In [6]:
import numpy as np
import scipy
from sympy import *
import random

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
plt.rcParams["figure.figsize"] = (8,6)

### Вспомогательная функция

In [3]:
def roots_to_dict(roots, x):
    dict_roots = dict()
    for i in range(len(roots)):
        dict_roots[x[i]] = roots[i]

    return dict_roots

### Якобиан

In [4]:
def get_jacobi(system_equations: np.array):
    n = system_equations.shape[0]
    x = symbols(f'x:{n}')
    J = np.empty(shape=(n, n), dtype=core.add.Add)
    for i in range(n):
        for j in range(n):
            J[i, j] = system_equations[i].diff(x[j])

    return J

### Метод простых итераций

In [7]:
def get_q(fi_equation, approx, e=0.1):
    x_0 = approx[0]
    x_fi = approx[1]
    x = symbols('x:2')
    e = 0.1
    x_1 = random.uniform(x_0 - e, x_0 + e)
    x_2 = random.uniform(x_0 - e, x_0 + e)

    q = (abs(fi_equation.subs({x[0]: x_1, x[1]: x_fi}) - fi_equation.subs({x[0]: x_2, x[1]: x_fi}))) / (abs(x_1 - x_2))
    return q

In [32]:
def iteration_solve(system_equations: np.array, approx, tol=0.00001, verbose=0):
    if verbose == 1:
        print('\niteration method computing...')

    n = system_equations.shape[0]
    x = symbols(f'x:{n}')

    fi_equations = system_equations[1]

    prev_roots = np.zeros(shape=(n, ))
    curr_roots = list(approx)

    errors = np.zeros(shape=(n, ))
    error = tol * 10000

    J = get_jacobi(system_equations[0])
    jacobi_values = np.zeros(shape=(n, n))

    roots_d = roots_to_dict(curr_roots, x)

    for i in range(n):
        for j in range(n):
            jacobi_values[i, j] = J[i, j].subs(roots_d)

    # compute q
    if verbose == 1:
        q_1 = float(get_q(fi_equations[0], (approx[0], approx[1]), 0.1))
        q_2 = float(get_q(fi_equations[1], (approx[1], approx[0]), 0.1))
        print(f'q_1 = {q_1}')
        print(f'q_2 = {q_2}')
        if q_1>=1 or q_2>=1:
            print("q is greater than 1")
            return None, None

    iteration = 0
    while error > tol:
        prev_roots = curr_roots.copy()
        roots_d = roots_to_dict(curr_roots, x)
        for i in range(n):
            try:
                curr_roots[i] = float(fi_equations[i].subs(roots_d))
            except TypeError:
                print("some complex numbers")

            errors[i] = abs(prev_roots[i] - curr_roots[i])
            roots_d = roots_to_dict(curr_roots, x)

        error = np.amax(errors)
        iteration += 1

    if verbose == 1:
        print('stopped iteration method\n')

    return curr_roots, iteration

### Метод Ньютона

In [9]:
import numpy as np
from sympy import symbols
from staff_matrix import get_jacobi, roots_to_dict


def newton_solve(system_equations: np.array, approx, tol=0.00001):
    n = system_equations.shape[0]
    x = symbols(f'x:{n}')

    J = get_jacobi(system_equations)

    error = tol * 10000
    iteration = 0
    roots = approx
    while error > tol:
        iteration += 1

        roots_d = roots_to_dict(roots, x)
        jacobi_values = np.zeros(shape=(n, n))
        for i in range(n):
            for j in range(n):
                jacobi_values[i, j] = J[i, j].subs(roots_d)

        jacobi_det = np.linalg.det(jacobi_values)
        print(f"Jacobi det = {jacobi_det}")
        if not jacobi_det:
            print("det equal 0. Can't solve system")
            exit(0)

        F = np.zeros(shape=(n, ))
        for i in range(0, n):
            F[i] = system_equations[i].subs(roots_d)

        delta_x = np.zeros(shape=(n, ), dtype=float)
        delta_x = np.linalg.solve(jacobi_values, -1 * F)

        roots = delta_x + roots
        error = np.amax(abs(delta_x))

    return roots, iteration

### Тестовый пример 1

In [10]:
x = symbols('x:2')

In [15]:
y_1_1 = 2*x[0]**2 - x[0]*x[1] - 5*x[0] + 1
y_1_2 = x[0] + 3*log(x[0], 10) - x[1]**2
approx = (3.5, 2.2)

In [12]:
def get_system_1_it():
    global y_1_1, y_1_2
    return np.array([
        [
            y_1_1,
            y_1_2
        ],
        [
            sqrt((x[0] * x[1] + 5*x[0] - 1) / 2),
            sqrt(x[0]+3*log(x[0], 10))
        ]
        ])

In [13]:
def print_list(ls, ):
    output = [el for el in ls]
    return output

In [14]:
system = get_system_1_it()

In [16]:
print(f"\t\tНачальное приближение: {approx}")
iteration_ans = iteration_solve(system, approx, verbose=1)
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (3.5, 2.2)

iteration method computing...
q_1 = 0.5158238578928235
q_2 = 0.4406675171821053
stopped iteration method

		*** Метод простой итерации: ***
Корни уравнения: [3.4874319608951048, 2.261625342754801]
Количество итераций: 14
--------------------
--------------------
		Начальное приближение: (3.5, 2.2)
Jacobi det = -25.11711655429025
Jacobi det = -25.48384528052109
Jacobi det = -25.46186954784371
		*** Метод Ньютона: ***
Корни уравнения: [3.4874427876429723, 2.261628630553625]
Количество итераций: 3


### Тестовый пример 2

In [17]:
approx = (1.8, -1.7)

In [19]:

print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1.8, -1.7)
Jacobi det = 16.362883445709755
Jacobi det = 10.188708287087627
Jacobi det = 9.046328946766074
Jacobi det = 8.997939217807113
		*** Метод Ньютона: ***
Корни уравнения: [1.4588902301577535, -1.3967670091993125]
Количество итераций: 4


In [20]:
print(f"\t\tНачальное приближение: {approx}")
iteration_ans = iteration_solve(system, approx, verbose=1)
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)

		Начальное приближение: (1.8, -1.7)

iteration method computing...
q_1 = 0.531618835427274


TypeError: Cannot convert complex to float

### Тестовый пример 3

In [24]:
y_2_1 = x[0]**2 + x[0]*x[1] - 10
y_2_2 = x[1] + 3*x[0]*x[1]**2 - 57
approx = (1.5, 3.5)


In [22]:
def get_system_2_it():
    global y_2_1, y_2_2
    return np.array([
        [
            y_2_1,
            y_2_2
        ],
        [
            sqrt(10 - x[0]*x[1]),
            sqrt((57-x[1]) / (3*x[0]))
        ]
    ])

In [25]:
system = get_system_2_it()
print(f"\t\tНачальное приближение: {approx}")
iteration_ans = iteration_solve(system, approx, verbose=1)
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1.5, 3.5)

iteration method computing...
q_1 = 0.8324715585096275
q_2 = 0.32696307217832404
stopped iteration method

		*** Метод простой итерации: ***
Корни уравнения: [1.9999986994971521, 3.0000010616488098]
Количество итераций: 12
--------------------
--------------------
		Начальное приближение: (1.5, 3.5)
Jacobi det = 156.12499999999994
Jacobi det = 197.78430344142228
Jacobi det = 204.96962918261596
Jacobi det = 204.9999473486533
		*** Метод Ньютона: ***
Корни уравнения: [1.9999999999999798, 3.000000000000075]
Количество итераций: 4


### Тестовый пример 4

In [27]:
y_3_1 = x[0]**2 - x[1] - 1
y_3_2 = x[0] - x[1]**2 + 1
approx = (1.9, 1.8)

In [28]:
def get_system_3_it():
    global y_3_1, y_3_2
    return np.array([
        [
            y_3_1,
            y_3_2
        ],
        [
            sqrt(1+x[1]),
            sqrt(1 + x[0])
        ]
    ])

In [29]:
system = get_system_3_it()
print(f"\t\tНачальное приближение: {approx}")
iteration_ans = iteration_solve(system, approx, verbose=1)
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1.9, 1.8)

iteration method computing...
q_1 = 0.0
q_2 = 0.29812530968953266
stopped iteration method

		*** Метод простой итерации: ***
Корни уравнения: [1.6180344244091438, 1.618034123376001]
Количество итераций: 6
--------------------
--------------------
		Начальное приближение: (1.9, 1.8)
Jacobi det = -12.679999999999998
Jacobi det = -9.741563554219864
Jacobi det = -9.474681428174463
Jacobi det = -9.472136189528126
		*** Метод Ньютона: ***
Корни уравнения: [1.6180339887498951, 1.618033988749895]
Количество итераций: 4


### Тестовый пример 5

In [35]:
y_4_1 = x[0] + x[0]*x[1] - 4
y_4_2 = x[0] + x[1] - 3
approx = (1.9, 0.6)
def get_system_4_it():
    global y_4_1, y_4_2
    return np.array([
        [
            y_4_1,
            y_4_2
        ],
        [
            x[0] + x[0]*x[1] - 4,
            x[0] + x[1] - 3
        ]
    ])

In [36]:
system = get_system_4_it()
print(f"\t\tНачальное приближение: {approx}")
iteration_ans = iteration_solve(system, approx, verbose=1)
print("\t\t*** Метод простой итерации: ***")
#print(f"Корни уравнения: {print_list(iteration_ans[0])}")
#print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1.9, 0.6)

iteration method computing...
q_1 = 1.5999999999999996
q_2 = 0.9999999999999991
q is greater than 1
		*** Метод простой итерации: ***
--------------------
--------------------
		Начальное приближение: (1.9, 0.6)
Jacobi det = -0.30000000000000004
Jacobi det = 0.2666666666666665
Jacobi det = 0.1333333333333356
Jacobi det = 0.06666666666666628
Jacobi det = 0.03333333333332781
Jacobi det = 0.016666666666655866
Jacobi det = 0.008333333333342475
Jacobi det = 0.004166666666719806
Jacobi det = 0.00208333333327293
Jacobi det = 0.0010416666665056574
Jacobi det = 0.0005208333332703521
Jacobi det = 0.00026041666538227805
Jacobi det = 0.0001302083316642588
Jacobi det = 6.510416258689651e-05
Jacobi det = 3.25520764255529e-05
		*** Метод Ньютона: ***
Корни уравнения: [1.9999918619811339, 1.0000081380188661]
Количество итераций: 15


### Тестовый пример 6

In [37]:
y_5_1 = x[0]**2 + x[0]*x[1] + x[1]**2 - 7
y_5_2 = x[0]**3 + x[1]**3 - 9
approx = (1.5, 0.9)
def get_system_5_it():
    global y_5_1, y_5_2
    return np.array([
        [
            y_5_1,
            y_5_2
        ],
        [
            x[0]**2 + x[0]*x[1] + x[1]**2 - 7,
            x[0]**3 + x[1]**3 - 9
        ]
    ])

In [38]:
system = get_system_5_it()
print(f"\t\tНачальное приближение: {approx}")
iteration_ans = iteration_solve(system, approx, verbose=1)
print("\t\t*** Метод простой итерации: ***")
#print(f"Корни уравнения: {print_list(iteration_ans[0])}")
#print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1.5, 0.9)

iteration method computing...
q_1 = 3.8806290962017433
q_2 = 2.7720778978549667
q is greater than 1
		*** Метод простой итерации: ***
--------------------
--------------------
		Начальное приближение: (1.5, 0.9)
Jacobi det = -12.797999999999998
Jacobi det = -49.51419979155356
Jacobi det = -35.804156196742795
Jacobi det = -33.116979273595206
Jacobi det = -33.00021634775366
		*** Метод Ньютона: ***
Корни уравнения: [2.0000000000117937, 0.9999999999882065]
Количество итераций: 5


### Задание

In [39]:
m = 0.2
a = 0.8

approx = (0.6, 0.6)

y_1 = tan(x[0]*x[1] + m) - x[0]
y_2 = a*x[0]**2 + 2*x[1]**2 - 1


def get_system():
    return np.array([
        [
            y_1,
            y_2
        ],
        [
            tan(x[0]*x[1] + m),
            sqrt((1 - a*x[0]**2) / 2)
        ]
    ])

In [40]:
system = get_system()
print(f"\t\tНачальное приближение: {approx}")
iteration_ans = iteration_solve(system, approx, verbose=1)
print("\t\t*** Метод простой итерации: ***")
print(f"Корни уравнения: {print_list(iteration_ans[0])}")
print(f"Количество итераций: {iteration_ans[1]}")

for i in range(2):
    print('-' * 20)


print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (0.6, 0.6)

iteration method computing...
q_1 = 0.8780753382488651
q_2 = 0.40178538780165235
stopped iteration method

		*** Метод простой итерации: ***
Корни уравнения: [0.6442300973695991, 0.5779161121281035]
Количество итераций: 10
--------------------
--------------------
		Начальное приближение: (0.6, 0.6)
Jacobi det = -1.1963912183495855
Jacobi det = -1.372482323529407
Jacobi det = -1.3609453316173887
Jacobi det = -1.3609293849707207
		*** Метод Ньютона: ***
Корни уравнения: [0.6442363542516425, 0.5779133221722967]
Количество итераций: 4


### Тестовый пример 6

In [43]:
y_6_1 = x[0]**2 - x[1] - 1
y_6_2 = 3*x[0]**2 - 3*x[1] - 3
approx = (1, 1)
def get_system_6_it():
    global y_6_1, y_6_2
    return np.array([
        [
            y_6_1,
            y_6_2
        ],
        [
            x[0]**2 - x[1] - 1,
            3*x[0]**2 - 3*x[1] - 3
        ]
    ])

In [46]:
system = get_system_6_it()


print(f"\t\tНачальное приближение: {approx}")
newton_ans = newton_solve(system[0], approx)
print("\t\t*** Метод Ньютона: ***")
print(f"Корни уравнения: {print_list(newton_ans[0])}")
print(f"Количество итераций: {newton_ans[1]}")

		Начальное приближение: (1, 1)
Jacobi det = 0.0
det equal 0. Can't solve system


LinAlgError: Singular matrix